In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('C:\\Users\\yu886\\OneDrive\\デスクトップ\\github\\Sony\\src')

from Processing.load_dataset import Load_dataset

from Engineering.dummy import engin_2, engin

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter('ignore', category=UserWarning)
import matplotlib.pyplot as plt
from Processing.processing import Submission

In [2]:
def add(df_train):
    df_train["co_avg"] = df_train["co_mid"] / df_train["co_cnt"]
    df_train["o3_avg"] = df_train["o3_mid"] / df_train["o3_cnt"]
    df_train["so2_avg"] = df_train["so2_mid"] / df_train["so2_cnt"]
    df_train["no2_avg"] = df_train["no2_mid"] / df_train["no2_cnt"]
    df_train["temperature_avg"] = df_train["temperature_mid"] / df_train["temperature_cnt"]
    df_train["humidity_avg"] = df_train["humidity_mid"] / df_train["humidity_cnt"]
    df_train["pressure_avg"] = df_train["pressure_mid"] / df_train["pressure_cnt"]
    df_train["ws_avg"] = df_train["ws_mid"] / df_train["ws_cnt"]
    df_train["dew_avg"] = df_train["dew_mid"] / df_train["dew_cnt"]
    return df_train

def add_2(df):
    df["discomfort"] = 0.81*df["temperature_mid"] + 0.01*df["humidity_mid"]*(0.99*df["temperature_mid"] - 14.3) + 46.3
    return df

def add_3(df):
    df["avg_count"] = (df_train["co_cnt"] + df_train["o3_cnt"] + df_train["so2_cnt"] + df_train["no2_cnt"] +
                       df_train["temperature_cnt"] + df_train["humidity_cnt"] + df_train["pressure_cnt"] + df_train["ws_cnt"] + df_train["dew_cnt"]) / 9
    return df

In [7]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

df_train = add(df_train)
df_test = add(df_test)

df_train = add_2(df_train)
df_test = add_2(df_test)

df_train = add_3(df_train)
df_test = add_3(df_test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [8]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 433.65964670210286 RMSE: 20.824496313286975 R^2: 0.6273265696251071 MAE: 15.144815952647726
fold 1 MSE: 490.4381543630323 RMSE: 22.145838307976337 R^2: 0.6555928430461975 MAE: 16.036445026494356
fold 2 MSE: 466.7595585954529 RMSE: 21.60461891807983 R^2: 0.6451460579103949 MAE: 15.529845632290147
fold 3 MSE: 516.1983216573208 RMSE: 22.719998275909283 R^2: 0.6583782319676745 MAE: 16.527023843550708
fold 4 MSE: 621.7459240717869 RMSE: 24.934833548106692 R^2: 0.6951277199706589 MAE: 16.70520602034777
CV score: 22.445957072671824


In [12]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [13]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 433.23946246968353 RMSE: 20.81440516732783 R^2: 0.6276876627092249 MAE: 15.134872706610516
fold 1 MSE: 484.79337842458307 RMSE: 22.01802394459101 R^2: 0.6595568519947416 MAE: 15.932107784315567
fold 2 MSE: 462.36774483543445 RMSE: 21.5027380776364 R^2: 0.6484849342054098 MAE: 15.457383564804662
fold 3 MSE: 508.1258808215025 RMSE: 22.541647695354982 R^2: 0.663720600191218 MAE: 16.4073247279028
fold 4 MSE: 613.3321222230056 RMSE: 24.765543043167973 R^2: 0.6992534164232453 MAE: 16.618186178742075
CV score: 22.32847158561564


In [7]:
# df_test = df_test.drop('City', axis=1)
pred_1 = models[0].predict(df_test)
pred_2 = models[1].predict(df_test)
pred_3 = models[2].predict(df_test)
pred_4 = models[3].predict(df_test)
pred_5 = models[4].predict(df_test)

pred = np.stack([pred_1, pred_2, pred_3 ,pred_4, pred_5], axis=1)
pred = np.mean(pred, axis=1)

index = test["id"]
Submission(index, pred, name='lgb_group_2')
pred

array([23.27969884, 39.04809959, 23.66257006, ..., 66.87592214,
       34.7394687 , 35.77972298])

In [2]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [4]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 433.2742088027439 RMSE: 20.815239820927932 R^2: 0.6276578028058815 MAE: 15.127949267609116
fold 1 MSE: 484.8196701743738 RMSE: 22.018620987118467 R^2: 0.6595383887762578 MAE: 15.933691388699774
fold 2 MSE: 462.8362746632746 RMSE: 21.513629974118142 R^2: 0.6481287344161737 MAE: 15.460176242064172
fold 3 MSE: 506.1455462154293 RMSE: 22.497678685042803 R^2: 0.6650311922273375 MAE: 16.385682186264603
fold 4 MSE: 611.8949709411876 RMSE: 24.736510888587087 R^2: 0.6999581216268846 MAE: 16.607792667936415
CV score: 22.316336071158887


In [10]:
# df_test = df_test.drop('City', axis=1)
pred_1 = models[0].predict(df_test)
pred_2 = models[1].predict(df_test)
pred_3 = models[2].predict(df_test)
pred_4 = models[3].predict(df_test)
pred_5 = models[4].predict(df_test)

pred = np.stack([pred_1, pred_2, pred_3 ,pred_4, pred_5], axis=1)
pred = np.mean(pred, axis=1)

index = test["id"]
Submission(index, pred, name='lgb_group_3')
pred

array([27.38051623, 38.83449391, 23.47738366, ..., 66.69708225,
       34.7743657 , 36.07596373])

In [7]:
df_train = add_2(df_train)
df_test = add_2(df_test)

df_train = add_3(df_train)
df_test = add_3(df_test)

In [8]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 433.016001535848 RMSE: 20.809036535501782 R^2: 0.6278796979917343 MAE: 15.125593533275458
fold 1 MSE: 485.64518627653797 RMSE: 22.0373588770646 R^2: 0.6589586752053689 MAE: 15.954829472971637
fold 2 MSE: 464.49595710189993 RMSE: 21.552168269153338 R^2: 0.6468669608860604 MAE: 15.483612288087237
fold 3 MSE: 511.1409750929699 RMSE: 22.608427081355526 R^2: 0.6617252007631537 MAE: 16.461441493225
fold 4 MSE: 618.4209698333672 RMSE: 24.868071292992692 R^2: 0.6967581068222874 MAE: 16.66996016907186
CV score: 22.37501241121359


In [3]:
import sklearn.preprocessing as preproc

In [4]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [5]:
pos = ['co_min', 'co_mid', 'co_max','o3_min',
       'o3_mid', 'o3_max', 'so2_min', 'so2_mid', 'so2_max',
       'no2_min', 'no2_mid', 'no2_max',
       'temperature_min', 'temperature_mid', 'temperature_max',
       'humidity_min', 'humidity_mid', 'humidity_max',
       'pressure_min', 'pressure_mid', 'pressure_max', 'ws_min',
       'ws_mid', 'ws_max', 'dew_min', 'dew_mid', 'dew_max']

X1 = df_train[pos]

# ペアワイズ交互作用特徴量
X2 = preproc.PolynomialFeatures(include_bias=False).fit_transform(X1)

In [6]:
print(f'生成後の列数: {X1.shape[1]}')
print(f'生成後の列数: {X2.shape[1]}')

生成後の列数: 27
生成後の列数: 405


In [7]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [12]:
df_train = df_train.drop(col, axis=1)
X2 = pd.DataFrame(X2).T

df_train = pd.concat([df_train, X2], axis=1)

Index(['City', 'co_cnt', 'o3_cnt', 'so2_cnt', 'no2_cnt', 'temperature_cnt',
       'humidity_cnt', 'pressure_cnt', 'ws_cnt', 'dew_cnt', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2019',
       'year_2020', 'year_2021', 'Australia', 'Belgium',
       'Bosnia and Herzegovina', 'Brazil', 'Canada', 'Chile', 'China',
       'Croatia', 'France', 'Germany', 'Hungary', 'India', 'Iran', 'Israel',
       'Italy', 'Japan', 'Mexico', 'Netherlands', 'Poland', 'Russia', 'Serbia',
       'South Africa', 'South Korea', 'Spain', 'Taiwan', 'Thailand', 'Turkey',
       'United Kingdom', 'United States', 'Vietnam', 'co_std', 'o3_std',
       'so2_std', 'no2_std', 'temperature_std', 'humidity_std', 'pressure_std',
       'ws_std', 'dew_std'],
      dtype='object')

In [15]:
X2 = pd.DataFrame(X2).T
df_train = pd.concat([df_train, X2], axis=1)

In [16]:
df_train

,City,co_cnt,o3_cnt,so2_cnt,no2_cnt,temperature_cnt,humidity_cnt,pressure_cnt,ws_cnt,dew_cnt,...,395,396,397,398,399,400,401,402,403,404
0,Brisbane,38,29,11,36,36,45,59,42,17,...,9.616201,23.787771,32.120158,46.862312,58.844241,79.456218,115.924152,107.288164,156.530096,228.372544
1,Darwin,47,49,31,45,71,50,69,48,62,...,54.700816,157.712304,176.120948,179.138516,454.712976,507.788412,516.488604,567.058969,576.774673,586.656841
2,Melbourne,17,123,36,111,148,138,153,125,22,...,65.431921,83.389501,106.232837,124.748558,106.275481,135.388097,158.985398,172.475689,202.537126,237.838084
3,Newcastle,63,90,50,176,101,111,141,52,116,...,12.902464,25.668432,38.380520,47.931648,51.065316,76.355010,95.356224,114.169225,142.580640,178.062336
4,Perth,47,83,47,80,55,155,124,102,93,...,11.532816,3.705036,11.128692,41.675712,1.190281,3.575207,13.388752,10.738729,40.215344,150.601984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,Jacksonville,12,26,8,26,40,48,55,22,12,...,37.515625,102.740750,138.908875,159.605250,281.367076,380.417546,437.096892,514.337041,590.969382,679.019364
195937,Las Vegas,14,8,9,31,51,55,67,51,12,...,47.073321,71.573952,101.138001,108.589047,108.826624,153.778112,165.107264,217.297081,233.305807,250.493929
195938,Milwaukee,171,112,120,66,170,91,194,125,26,...,6.646084,5.282322,9.102918,17.236508,4.198401,7.235019,13.699614,12.467961,23.608266,44.702596
195939,Hanoi,31,108,27,149,290,260,293,266,51,...,36.060025,11.541610,44.695215,46.334580,3.694084,14.305446,14.830152,55.398249,57.430188,59.536656


In [17]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 435.94206904793316 RMSE: 20.879225777023752 R^2: 0.625365127808573 MAE: 15.183887993237695


KeyboardInterrupt: 

# XGBOOST

In [ ]:
FOLD = 10
NUM_ROUND = 2000
VERBOSE_EVAL = -1

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate':0.01,
    'max_depth':8,
}

valid_scores = []
models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_valid, label=y_valid)
    evals = [(dtrain, 'train'), (dtest, 'eval')]

    model = xgb.train(params,
                      dtrain,
                      num_boost_round=NUM_ROUND,
                      evals=evals,
                      early_stopping_rounds=20,
                      verbose_eval=200
                      )

    dm_valid = xgb.DMatrix(X_valid)
    y_valid_pred = model.predict(dm_valid)

    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')